In [1]:
import torch
import flair
from flair.data import Sentence

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [45]:
import io
import pandas as pd
data = pd.read_csv('train_E6oV3lV.csv') 
data.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [46]:
text = data['tweet']
txt = text.tolist()
print(txt[:5])

[' @user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction.   #run', "@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx.    #disapointed #getthanked", '  bihday your majesty', '#model   i love u take with u all the time in urð\x9f\x93±!!! ð\x9f\x98\x99ð\x9f\x98\x8eð\x9f\x91\x84ð\x9f\x91\x85ð\x9f\x92¦ð\x9f\x92¦ð\x9f\x92¦  ', ' factsguide: society now    #motivation']


In [47]:
from flair.embeddings import WordEmbeddings
from flair.embeddings import CharacterEmbeddings
from flair.embeddings import StackedEmbeddings
from flair.embeddings import FlairEmbeddings
from flair.embeddings import BertEmbeddings
from flair.embeddings import ELMoEmbeddings


### Initialising embeddings (un-comment to use others) ###

glove_embedding = WordEmbeddings('glove')
character_embeddings = CharacterEmbeddings()
flair_forward  = FlairEmbeddings('news-forward-fast')
flair_backward = FlairEmbeddings('news-backward-fast')
bert_embedding = BertEmbeddings()


stacked_embeddings = StackedEmbeddings( embeddings = [ glove_embedding])

In [48]:
from flair.data import Sentence
sentence = Sentence('This is Glove Embedding')

stacked_embeddings.embed(sentence)


for token in sentence:
    print(token.embedding)
    print(type(token.embedding))
# storing size (length) #
z = token.embedding.size()[0]

tensor([-0.5706,  0.4418,  0.7010, -0.4171, -0.3406,  0.0234, -0.0715,  0.4818,
        -0.0131,  0.1683, -0.1339,  0.0406,  0.1583, -0.4434, -0.0194, -0.0097,
        -0.0463,  0.0932, -0.2733,  0.2285,  0.3309, -0.3647,  0.0787,  0.3585,
         0.4476, -0.2299,  0.1808, -0.6265,  0.0539, -0.2915, -0.4256,  0.6290,
         0.1439, -0.0460, -0.2101,  0.4888, -0.0577,  0.3743, -0.0301, -0.3449,
        -0.2970,  0.1509,  0.2825, -0.1658,  0.0761, -0.0930,  0.7936, -0.6049,
        -0.1887, -1.0173,  0.3196, -0.1634,  0.5418,  1.1725, -0.4787, -3.3842,
        -0.0813, -0.3528,  1.8372,  0.4452, -0.5267,  0.9979, -0.3218,  0.0335,
         1.1783, -0.0729,  0.3974,  0.2617,  0.3311, -0.3563, -0.1656, -0.4438,
        -0.1418, -0.3798,  0.2899, -0.0291, -0.3517, -0.2769, -1.3440,  0.1955,
         0.1689,  0.0402, -0.8021,  0.2337, -1.3837, -0.0231,  0.0854, -0.7405,
        -0.0739, -0.5884, -0.0857, -0.1053, -0.5157,  0.1504, -0.1669, -0.1637,
        -0.2270, -0.6610,  0.4720,  0.37

In [49]:
from tqdm import tqdm ## tracks progress of loop ##

# creating a tensor for storing sentence embeddings #
s = torch.zeros(0,z)

# iterating Sentence (tqdm tracks progress) #


for q in tqdm(txt):
    w = torch.zeros(0,z)
    sentence = Sentence(q)
    stacked_embeddings.embed(sentence)
    for token in sentence:
        w = torch.cat((w,token.embedding.view(-1,z)),0)
        s = torch.cat((s, w.mean(dim = 0).view(-1, z)),0)

100%|██████████████████████████████████████████████████████████████████████████| 31962/31962 [5:47:59<00:00,  1.01s/it]


In [53]:
from flair.embeddings import DocumentPoolEmbeddings


document_embeddings = DocumentPoolEmbeddings([glove_embedding ])

z = sentence.embedding.size()[0]

s = torch.zeros(0,z)

for tweet in tqdm(txt):
    sentence = Sentence(tweet)
    document_embeddings.embed(sentence)
    s = torch.cat((s, sentence.embedding.view(-1,z)),0)

torch.save(document_embeddings, 'glove_tweet_embed.pt')

100%|██████████████████████████████████████████████████████████████████████████| 31962/31962 [5:34:28<00:00,  1.59it/s]


In [38]:

loaded_embeddings: DocumentPoolEmbeddings = torch.load('gloveembedding.pt')
print(loaded_embeddings)

DocumentPoolEmbeddings(
  fine_tune_mode=linear, pooling=mean
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings('glove')
  )
  (embedding_flex): Linear(in_features=100, out_features=100, bias=False)
)


In [39]:
sentence = Sentence('when a saw a car run over')
loaded_embeddings.embed(sentence)

# print embeddings
for token in sentence:
    print(token.embedding[:10])

tensor([ 0.0732,  0.1113,  0.6228, -0.3591, -0.7073,  0.4376,  0.1282,  0.1348,
         0.3428, -0.3166])
tensor([-0.2709,  0.0440, -0.0203, -0.1740,  0.6444,  0.7121,  0.3551,  0.4714,
        -0.2964,  0.5443])
tensor([ 0.1604,  0.0303, -0.2871, -0.1826, -0.4708, -0.0641, -0.0619,  0.2830,
         0.1546, -0.4104])
tensor([-0.2709,  0.0440, -0.0203, -0.1740,  0.6444,  0.7121,  0.3551,  0.4714,
        -0.2964,  0.5443])
tensor([-0.1684, -0.5383,  0.3115, -0.5322,  0.2668, -0.1364,  0.3662,  0.6838,
         0.7773,  0.6805])
tensor([-0.0833,  0.2392,  0.0788, -0.7250,  0.5295,  0.3875,  0.0230,  0.4968,
        -0.4881, -0.0962])
tensor([-0.2957,  0.3535,  0.6333,  0.1958, -0.0303,  0.5424, -0.2109,  0.3289,
        -0.4889,  0.1838])


In [41]:
from flair.models import TextClassifier
from flair.trainers import ModelTrainer


classifier = TextClassifier.load('gloveembedding.pt')

# create example sentence
sentence = Sentence('when a saw a car run over')

# predict tags and print
classifier.predict(sentence)

print(sentence.label)

2019-07-18 20:06:14,160 loading file gloveembedding.pt


TypeError: 'DocumentPoolEmbeddings' object is not subscriptable

In [42]:
## tensor to numpy array ##
import numpy as np

##df = pd.read_csv('C:/Dataset.csv')
#data['split'] = np.random.randn(data.shape[0], 1)

#msk = np.random.rand(len(data)) <= 0.5

#train = data[msk]
#test = data[~msk]




X = s.detach().numpy()   

## Test set ##
test = X[5001:,:]
train = X[:5001,:]

print(train.shape)
print(test.shape)

# extracting labels of the training set #
target = data['label'].values
#target = data['label'][data['label'].isnull()==False].values

(5001, 100)
(5000, 100)


In [43]:
def custom_eval(preds, dtrain):
    labels = dtrain.get_label().astype(np.int)
    preds = (preds >= 0.3).astype(np.int)
    return [('f1_score', f1_score(labels, preds))]

In [44]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

### Splitting training set ###
x_train, x_valid, y_train, y_valid = train_test_split(train, target,  
                                                      random_state=42, 
                                                          test_size=0.3)

### XGBoost compatible data ###
dtrain = xgb.DMatrix(x_train,y_train)         
dvalid = xgb.DMatrix(x_valid, label = y_valid)

watchlist = [(dtrain, 'train'), (dvalid, 'valid')]


### defining parameters ###
params = {
          'colsample': 0.9,
          'colsample_bytree': 0.5,
          'eta': 0.1,
          'max_depth': 8,
          'min_child_weight': 6,
          'objective': 'multi:softmax',
          'num_class':6,
          'subsample': 0.9
          }

### Training the model ###
xgb_model = xgb.train(
                      params,
                      dtrain,
                      feval= custom_eval,
                      num_boost_round= 1000,
                      maximize=True,
                    #  evals=[(dvalid, "Validation")],
                      evals= watchlist,
                      early_stopping_rounds=30
                      )

ValueError: Found input variables with inconsistent numbers of samples: [5001, 10001]

In [ ]:
### Reformatting test set for XGB ###
dtest = xgb.DMatrix(test)

### Predicting ###
predict = xgb_model.predict(dtest) # predicting